In [ ]:

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

def load_data(data_path, vocab_size):
  with open(data_path, encoding = 'latin1') as f:
    d_lines = f.read().splitlines()
  data,labels = [],[]
  for data_id, line in enumerate(d_lines):
    vector = [0.0 for _ in range(vocab_size)]
    features = line.split('<fff>')
    label, doc_id = int(features[0]), int(features[1])
    for token in features[2].split():
      index, value = int(token.split(':')[0]), float(token.split(':')[1])
      vector[index] = value
    data.append(vector)
    labels.append(label)
  return torch.tensor(data), torch.tensor(labels)

with open('/home/hoangntbn/Desktop/20192/project2/upserver/words_idf.txt', encoding = 'latin1') as f:
  vocab_size = len(f.read().splitlines())
X_train, Y_train = load_data(
    '/home/hoangntbn/Desktop/20192/project2/upserver/train_tf_idf_vector.txt', vocab_size)
X_test,Y_test = load_data(
    '/home/hoangntbn/Desktop/20192/project2/upserver/test_tf_idf_vector.txt', vocab_size)

In [ ]:


class NeuronNetwork(torch.nn.Module):
  def __init__(self, vocab_size, hidden_size, num_classes):
    super().__init__()
    self._vocab_size = vocab_size
    self._hidden_size = hidden_size
    self._num_classes = num_classes

  def build_graph(self):
    self._hidden_layer = torch.nn.Linear(self._vocab_size, self._hidden_size)
    self._output_layer = torch.nn.Linear(self._hidden_size, self._num_classes)

  def forward(self,x):
    x = torch.sigmoid(self._hidden_layer(x))
    x = F.softmax(self._output_layer(x), dim = 1)
    return x

  def fit(self, X_train, Y_train, batch_size, max_epochs=50 ,learning_rate=1e-2, threshold=1e-3):
    self.build_graph()
    dataset = TensorDataset(X_train, Y_train)
    data_loader = DataLoader(dataset, batch_size = batch_size, shuffle = True)
    opt = torch.optim.Adam(params = self.parameters(), lr = learning_rate)
    last_loss = 0
    for epoch in range(max_epochs):
      new_loss = 0
      for data,labels in data_loader:
        self.zero_grad()
       
   
        loss = F.cross_entropy(prediced, labels)
        new_loss += loss
       
        loss.backward()

        opt.step()
      new_loss = new_loss / len(data_loader)
      print('iter: {}, loss: {}'.format(epoch, new_loss))
      if abs(last_loss - new_loss) <= threshold:
        return
      last_loss=new_loss

  def predict(self, X):
    return torch.argmax(self.forward(X), dim = 1)

  def compute_accuracy(self,predicted,expected):
    return (predicted == expected).float().mean()

In [ ]:
from time import time
t=time()
NN = NeuronNetwork(
    vocab_size = vocab_size,
    hidden_size = 30,
    num_classes = 20
    )
NN.fit(
    X_train = X_train,
    Y_train = Y_train,
    batch_size = 32,
    max_epochs = 50,
    learning_rate = 1e-2,
    threshold = 1e-3
    )
print('training time:',time()-t,'s')
predicted = NN.predict(X_train)
print('train accuracy:', NN.compute_accuracy(predicted, Y_train))
predicted = NN.predict(X_test)
print('test accuracy:', NN.compute_accuracy(predicted, Y_test))